# Data Exploration - Hugging Face Stocks Dataset

Цел на този notebook:
- Зареждане на `paperswithbacktest/Stocks-Daily-Price` dataset от Hugging Face
- Преглед на структурата на данните
- Анализ на тикери и временен обхват
- Базова статистика и визуализации


In [1]:
# Импортиране на библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import warnings

warnings.filterwarnings('ignore')

# Настройка за визуализация
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Библиотеки заредени успешно!")


## 1. Зареждане на Dataset от Hugging Face


In [2]:
# Зареждане на dataset
DATASET_NAME = "paperswithbacktest/Stocks-Daily-Price"

print(f"Зареждане на dataset: {DATASET_NAME}...")
print("Това може да отнеме няколко минути при първо зареждане...")

# Зареждаме dataset-а
dataset = load_dataset(DATASET_NAME)

print(f"\nDataset зареден успешно!")
print(f"Split-ове: {list(dataset.keys())}")


## 2. Преглед на структурата на данните


In [3]:
# Преглед на структурата
print("Информация за dataset-а:")
print(f"Брой split-ове: {len(dataset)}")
print(f"\nПърви split: {list(dataset.keys())[0]}")
print(f"Брой редове в първия split: {len(dataset[list(dataset.keys())[0]])}")

# Погледни първия пример
first_split = list(dataset.keys())[0]
print(f"\nПримерна структура:")
print(dataset[first_split][0])


In [4]:
# Конвертиране в pandas DataFrame за по-лесна работа
# Взимаме първия split (обикновено е 'train' или 'default')
split_name = list(dataset.keys())[0]
df = dataset[split_name].to_pandas()

print(f"DataFrame създаден успешно!")
print(f"Размери: {df.shape}")
print(f"\nКолони:")
print(df.columns.tolist())


In [5]:
# Първите 10 реда
print("Първи 10 реда от dataset-а:")
df.head(10)


In [6]:
# Базова информация за данните
print("Базова информация:")
print(df.info())

print("\n" + "="*50)
print("Описателна статистика:")
print(df.describe())


## 3. Анализ на Тикери


In [7]:
# Уникални тикери
unique_tickers = df['symbol'].unique() if 'symbol' in df.columns else df.columns[0]

if isinstance(unique_tickers, str):
    # Ако колоната се казва различно, опитай да намерим тикер колоната
    ticker_cols = [col for col in df.columns if 'symbol' in col.lower() or 'ticker' in col.lower()]
    if ticker_cols:
        unique_tickers = df[ticker_cols[0]].unique()
    else:
        print("Не можах да намеря колона за тикери. Колоните са:")
        print(df.columns.tolist())
        unique_tickers = []

print(f"Брой уникални тикери: {len(unique_tickers)}")
print(f"\nПърви 20 тикера:")
print(unique_tickers[:20])


In [8]:
# Проверка за популярни тикери (ако има данни)
popular_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA']

if len(unique_tickers) > 0:
    # Проверяваме кои от популярните са в dataset-а
    available_popular = [tick for tick in popular_tickers if tick in unique_tickers]
    print(f"Популярни тикери налични в dataset-а: {available_popular}")
    print(f"\nОбщо {len(available_popular)}/{len(popular_tickers)} популярни тикера са налични")


## 4. Анализ на Времеви Период


In [9]:
# Проверка за date колоната
date_cols = [col for col in df.columns if 'date' in col.lower()]
print(f"Дата колони: {date_cols}")

if date_cols:
    date_col = date_cols[0]
    # Конвертиране в datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    print(f"\nПериод на данните:")
    print(f"Най-ранна дата: {df[date_col].min()}")
    print(f"Най-късна дата: {df[date_col].max()}")
    print(f"Обхват: {(df[date_col].max() - df[date_col].min()).days} дни")
else:
    print("Не можах да намеря date колона")


## 5. Проверка за Липсващи Стойности


In [10]:
# Липсващи стойности
print("Липсващи стойности по колони:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing Count', ascending=False)

print(missing_df[missing_df['Missing Count'] > 0])


## 6. Примерна Визуализация (за един тикер)


In [11]:
# Визуализация на данни за един тикер (ако има подходящи данни)
# Това ще зависи от структурата на dataset-а

# Опитваме се да намерим close или price колона
price_cols = [col for col in df.columns if 'close' in col.lower() or 'price' in col.lower()]

if price_cols and date_cols and len(unique_tickers) > 0:
    # Избираме първия наличен тикер
    sample_ticker = unique_tickers[0] if len(unique_tickers) > 0 else None
    
    if sample_ticker:
        ticker_col = 'symbol' if 'symbol' in df.columns else ticker_cols[0]
        ticker_data = df[df[ticker_col] == sample_ticker].copy()
        ticker_data = ticker_data.sort_values(date_cols[0])
        
        if len(ticker_data) > 0:
            plt.figure(figsize=(12, 6))
            plt.plot(ticker_data[date_cols[0]], ticker_data[price_cols[0]], linewidth=1.5)
            plt.title(f'Цена на акция {sample_ticker} през времето', fontsize=14)
            plt.xlabel('Дата')
            plt.ylabel('Цена')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        else:
            print("Няма данни за визуализация")
    else:
        print("Няма наличен тикер за визуализация")
else:
    print("Не можах да намеря подходящи колони за визуализация")
    print(f"Price колони: {price_cols}")
    print(f"Date колони: {date_cols}")


## Заключение

- Размер на dataset: {размери}
- Брой тикери: {брой}
- Период: {период}
- Следващи стъпки: Преглед на структурата и започване на предобработка
